# PZQ resistance: genome-wide association study


## Aim

This notebook describes the different analyses related to mapping the QTL of praziquantel (PZQ) resistance. We perform a genome with association study on a population of *Schistosoma mansoni* (SmLE-PZQ-R) which is polymorphic for PZQ resistance ([Couto *et al.*](https://doi.org/10.1590/s0074-02762011000200006)). The population was treated with a dose of 24 µg/mL of PZQ and survival of the worms was objectively assessed using a lactacte assay as a proxy. Pools of the 20% low and 20% high lactacte producer were made. The GWAS was conducted with two independent biological replicate and each library was sequenced in triplicate for assessing allele frequency.

After identifying the QTL related to resistance, we performed a marker assisted selection to create two populations: an enriched population in resistance allele (ER) and an enriched population in sensitive allele (ES).



## Environment and data

Installation of the environment

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run.

In [1]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate PZQ-R_DNA

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

(PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) 

: 1

The cell below must be run only once at the time of the environment creation.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Phenotypic data

The phenotypic data for the F0 parental population, the F1s and F2s is available on [Zenodo](https://doi.org/10.5281/zenodo.4383248). Data is already present in the repository but can be downloaded if needed.

In [ ]:
# Data directory
pdir="data/phenotypes"
[[ ! -d "$pdir" ]] && mkdir -p "$pdir"

# Download files
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F0_parental_populations.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F1.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F2.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/sex.tsv

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=   ## !! To UPDATE PRJNA667697

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    fastq-dump -O "$ldir/$fln/" --split-files "$run"
    
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo

### Genome data

The genome data is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14). The data is then indexed for the different tools used.

In [10]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

(PZQ-R_DNA) (PZQ-R_DNA) 

: 1

In [1]:
# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

[bwa_index] fail to open file '/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa' : No such file or directory
[fai_build] fail to open the FASTA file /schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa


### snpEff database

To annotate the functional consequences of the variants in the VCF file, we use the snpEff tool. Because the reference genome is recent, we have to build a snpEff database in order to annotate the variants. 

In [ ]:
# Create snpEff folder
sedir="$gdir/snpEff/"
mkdir "$sedir"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz

pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

# Convert gff to gtf to make it easier for snpEff
gffread "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -T -o "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf"

# Custom snpEff config file
cp "$CONDA_PREFIX"/share/snpeff-4.3.1t-4/snpEff.config "$sedir"

## Identify mitochondrial chromosome
mito=$(grep -i ">.*mito.*" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" | sed -r "s/>(.*) .*/\1/")

##  Add schistosome genome and specify mitochondrial chromosome
sed -i -r "s|data.dir = ./data/|data.dir = .| ; s|(# Ebola virus)|# S. mansoni\nsmansoni.genome: Schistosoma_mansoni\n\tsmansoni.$mito.codonTable : Trematode_Mitochondrial\n\n\1|" "$sedir"/snpEff.config

# Create the mandatory folder and link to the needed files
mkdir "$sedir"/genomes
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" "$PWD/$sedir/genomes/smansoni.fa"

mkdir "$sedir"/smansoni
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" "$PWD/$sedir/smansoni/genes.gtf"

# Generate the database
snpEff build -v -c "$sedir/snpEff.config" smansoni

### Known variants

List of variants from the SmLE x SmHR crosses that segregated in a Mendelian fashion in F1 ([Valentim *et al.* 2013](https://doi.org/10.1126/science.1243106)). This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

## Phenotype

ADD WINKA DATA

## Sequencing data processing

### Snakemake pipeline

The snakemake pipeline performs the following steps:
* alignment of each library against the reference genome
* marking duplicates
* base quality recalibration
* generating alignment statistics
* calling variants (gVCF) for each library
* combining gVCF of all libraries
* genotyping all samples per contig
* merging all the contigs to generate final VCF file

The VCF file produced will then be used to identify possible QTLs link to the cercarial production.

In [ ]:
# Status directory
statdir=status
[[ ! -d "$statdir" ]] && mkdir -p "$statdir"

snakemake --snakefile GWAS.smk --cluster "qsub -V -cwd -o $statdir -j y -r y -pe smp 10 -S /bin/bash" --jobs 384 -w 120

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cluster nodes: 384
Job counts:
	count	jobs
	12	alignment
	1	all
	12	bsqr
	12	calling
	2	combining
	320	genotype_variants
	12	indexing1
	12	indexing2
	12	indexing3
	12	mark_duplicates
	1	merge_variants
	12	stats
	420

[Sun Jan 17 19:25:51 2021]
rule alignment:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_2_WGS/SmLE-PZQ-R_Exp1_Recovered_2_WGS_R1.fastq.gz, data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_2_WGS/SmLE-PZQ-R_Exp1_Recovered_2_WGS_R2.fastq.gz, data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_2_WGS/SmLE-PZQ-R_Exp1_Recovered_2_WGS_sorted.bam
    jobid: 3
    wildcards: sample=SmLE-PZQ-R_Exp1_Recovered_2_WGS

Submitted job 3 with external jobid 'Your job 28328 ("snakejob.alignment.3.sh") has been submitted'.

[Sun Jan 17 19:25:52 2021]
rule alignment:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_2_WGS/SmLE-PZQ-R_Exp2_Contra

    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted.bam.bai
    jobid: 18
    wildcards: sample=SmLE-PZQ-R_Exp2_Contracted_1_WGS

Submitted job 18 with external jobid 'Your job 28550 ("snakejob.indexing1.18.sh") has been submitted'.
[Mon Jan 18 08:05:36 2021]
Finished job 18.
2 of 420 steps (0.48%) done

[Mon Jan 18 08:05:36 2021]
rule mark_duplicates:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted.bam.bai
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted_MD.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted_MD.log
    jobid: 30
    wildcards: sample=SmLE-PZQ-R_Exp2_Contracted_1_WGS

Submitted job 30 with external jobid 'Your job 28551 ("snakejob.mark_duplicates.30.sh") h

    jobid: 23
    wildcards: sample=SmLE-PZQ-R_Exp1_Recovered_1_WGS

Submitted job 23 with external jobid 'Your job 28561 ("snakejob.indexing1.23.sh") has been submitted'.
[Mon Jan 18 10:07:22 2021]
Finished job 5.
13 of 420 steps (3%) done

[Mon Jan 18 10:07:22 2021]
rule indexing1:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted.bam
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted.bam.bai
    jobid: 17
    wildcards: sample=SmLE-PZQ-R_Exp1_Contracted_2_WGS

Submitted job 17 with external jobid 'Your job 28562 ("snakejob.indexing1.17.sh") has been submitted'.
[Mon Jan 18 10:13:58 2021]
Finished job 48.
14 of 420 steps (3%) done

[Mon Jan 18 10:13:58 2021]
rule bsqr:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_3_WGS/SmLE-PZQ-R_Exp2_Recovered_3_WGS_sorted_MD.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_3_WGS/SmLE-PZQ-R_Exp2_Recovered_3_WGS_sorted_MD.


Submitted job 79 with external jobid 'Your job 28572 ("snakejob.stats.79.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_3_WGS/SmLE-PZQ-R_Exp2_Contracted_3_WGS_sorted_MD_recal.flagstat.
[Mon Jan 18 10:51:48 2021]
Finished job 79.
23 of 420 steps (5%) done
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted_MD.log.
[Mon Jan 18 10:56:58 2021]
Finished job 29.
24 of 420 steps (6%) done

[Mon Jan 18 10:56:58 2021]
rule indexing2:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted_MD.bam
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted_MD.bam.bai
    jobid: 41
    wildcards: sample=SmLE-PZQ-R_Exp1_Contracted_2_WGS

Submitted job 41 with external jobid 'Your job 28573 ("snakejob.indexing2.41.sh") has been submitted'.
Write-protecting output file data/lib

    jobid: 77
    wildcards: sample=SmLE-PZQ-R_Exp1_Contracted_2_WGS

Submitted job 77 with external jobid 'Your job 28582 ("snakejob.stats.77.sh") has been submitted'.

[Mon Jan 18 12:15:34 2021]
rule calling:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted_MD_recal.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS_sorted_MD_recal.bam.bai, data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa, data/genome/sm_dbSNP_v7.vcf
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_2_WGS/SmLE-PZQ-R_Exp1_Contracted_2_WGS.gvcf.gz
    jobid: 89
    wildcards: sample=SmLE-PZQ-R_Exp1_Contracted_2_WGS

Submitted job 89 with external jobid 'Your job 28583 ("snakejob.calling.89.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_1_WGS/SmLE-PZQ-R_Exp1_Recovered_1_WGS_sorted_MD_recal.bam.
[Mon Jan 18 12:17:45 2021]
Finished job 59.
34 of 4

    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Contracted_1_WGS/SmLE-PZQ-R_Exp2_Contracted_1_WGS_sorted_MD_recal.flagstat
    jobid: 78
    wildcards: sample=SmLE-PZQ-R_Exp2_Contracted_1_WGS

Submitted job 78 with external jobid 'Your job 28593 ("snakejob.stats.78.sh") has been submitted'.
[Mon Jan 18 12:46:47 2021]
Finished job 10.
44 of 420 steps (10%) done

[Mon Jan 18 12:46:47 2021]
rule indexing1:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_1_WGS/SmLE-PZQ-R_Exp2_Recovered_1_WGS_sorted.bam
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_1_WGS/SmLE-PZQ-R_Exp2_Recovered_1_WGS_sorted.bam.bai
    jobid: 22
    wildcards: sample=SmLE-PZQ-R_Exp2_Recovered_1_WGS

Submitted job 22 with external jobid 'Your job 28594 ("snakejob.indexing1.22.sh") has been submitted'.
[Mon Jan 18 12:48:37 2021]
Finished job 13.
45 of 420 steps (11%) done

[Mon Jan 18 12:48:37 2021]
rule mark_duplicates:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_3_WGS/SmLE-PZQ-R_E

    jobid: 28
    wildcards: sample=SmLE-PZQ-R_Exp2_Recovered_2_WGS

Submitted job 28 with external jobid 'Your job 28603 ("snakejob.mark_duplicates.28.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_3_WGS/SmLE-PZQ-R_Exp2_Recovered_3_WGS_sorted_MD_recal.bam.
[Mon Jan 18 13:28:18 2021]
Finished job 60.
55 of 420 steps (13%) done

[Mon Jan 18 13:28:18 2021]
rule indexing3:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_3_WGS/SmLE-PZQ-R_Exp2_Recovered_3_WGS_sorted_MD_recal.bam
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_3_WGS/SmLE-PZQ-R_Exp2_Recovered_3_WGS_sorted_MD_recal.bam.bai
    jobid: 72
    wildcards: sample=SmLE-PZQ-R_Exp2_Recovered_3_WGS

Submitted job 72 with external jobid 'Your job 28604 ("snakejob.indexing3.72.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_3_WGS/SmLE-PZQ-R_Exp1_Recovered_3_WGS_sorted_MD.log.
[Mon Jan 18 13:30:08 2021]
Fin

[Mon Jan 18 16:28:55 2021]
Finished job 38.
65 of 420 steps (15%) done

[Mon Jan 18 16:28:55 2021]
rule bsqr:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD.bam.bai, data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa, data/genome/sm_dbSNP_v7.vcf
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD.grp, data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD_recal.bam
    jobid: 50
    wildcards: sample=SmLE-PZQ-R_Exp1_Contracted_1_WGS

Submitted job 50 with external jobid 'Your job 29535 ("snakejob.bsqr.50.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Recovered_3_WGS/SmLE-PZQ-R_Exp1_Recovered_3_WGS_sorted_MD_recal.bam.
[Mon Jan 18 16:32:12 2021]
Finished job 49.
66 of 42

    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp2_Recovered_2_WGS/SmLE-PZQ-R_Exp2_Recovered_2_WGS.gvcf.gz
    jobid: 88
    wildcards: sample=SmLE-PZQ-R_Exp2_Recovered_2_WGS

Submitted job 88 with external jobid 'Your job 29564 ("snakejob.calling.88.sh") has been submitted'.
Write-protecting output file data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD_recal.bam.bai.
[Mon Jan 18 18:00:45 2021]
Finished job 62.
74 of 420 steps (18%) done

[Mon Jan 18 18:00:45 2021]
rule calling:
    input: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD_recal.bam, data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS_sorted_MD_recal.bam.bai, data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa, data/genome/sm_dbSNP_v7.vcf
    output: data/libraries/1-GWAS/SmLE-PZQ-R_Exp1_Contracted_1_WGS/SmLE-PZQ-R_Exp1_Contracted_1_WGS.gvcf.gz
    jobid: 86
    wildcards: sample=Sm

### Variant annotation

Functional consequences of variants is annotated to the VCF file using snpEff.

In [20]:
grep -m 1 VCF_PFX GWAS.smk | cut -d "\"" -f 2

PZQ_GWAS
(PZQ-R_DNA) 

: 1

In [21]:
# Status directory
cdir="data/calling"
[[ ! -d "$cdir" ]] && mkdir -p "$cdir"

myvcf=$(grep -m 1 VCF_PFX GWAS.smk | cut -d "\"" -f 2)

snpEff -v -c "$sedir/snpEff.config" smansoni "$cdir/$myvcf.vcf.gz" | bgzip -c -@ $(nproc) > "$cdir/${myvcf}_snpEff.vcf.gz"

# Index the VCF file
tabix "$cdir/${myvcf}_snpEff.vcf.gz"

(PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) 00:00:00	SnpEff version SnpEff 4.3t (build 2017-11-24 10:18), by Pablo Cingolani
00:00:00	Command: 'ann'
00:00:00	Reading configuration file 'data/genome/snpEff//snpEff.config'. Genome: 'smansoni'
00:00:00	Reading config file: /data/infectious/schistosome/06 - PZQ resistance/1-PZQ-DNA/data/genome/snpEff/snpEff.config
00:00:01	done
00:00:01	Reading database for genome version 'smansoni' from file '/data/infectious/schistosome/06 - PZQ resistance/1-PZQ-DNA/data/genome/snpEff/./smansoni/snpEffectPredictor.bin' (this might take a while)
00:00:06	done
00:00:06	Loading Motifs and PWMs
00:00:06	Building interval forest
00:00:06	Gene 'gene:Smp_320850' spans across coordinate zero: Assuming circular chromosome, creating mirror gene at the end.
	Gene        :Gene_SM_V7_2H012:-7145-7773
	New gene    :Gene_SM_V7_2H012:34721-49639
	Chrsomosome :Chromosome_SM_V7_2H012:1-41866
00:00:06	Total: 1 added as circular mirrored genes (a

#		'SM_V7_2H017'	35746	Standard
#		'SM_V7_2H019'	35649	Standard
#		'SM_V7_1H025'	35346	Standard
#		'SM_V7_1H024'	35172	Standard
#		'SM_V7_4H022'	35161	Standard
#		'SM_V7_ZWU007'	35161	Standard
#		'SM_V7_4H023'	34814	Standard
#		'SM_V7_3H012'	34678	Standard
#		'SM_V7_ZWH023'	34659	Standard
#		'SM_V7_4H024'	34614	Standard
#		'SM_V7_1H028'	34539	Standard
#		'SM_V7_ZWH022'	34526	Standard
#		'SM_V7_1003'	34508	Standard
#		'SM_V7_1H026'	34385	Standard
#		'SM_V7_1H027'	34385	Standard
#		'SM_V7_7H007'	34225	Standard
#		'SM_V7_4H025'	34158	Standard
#		'SM_V7_U012'	34139	Standard
#		'SM_V7_1H029'	34050	Standard
#		'SM_V7_5H018'	33970	Standard
#		'SM_V7_5H019'	33947	Standard
#		'SM_V7_2H020'	33720	Standard
#		'SM_V7_2H021'	33608	Standard
#		'SM_V7_6H015'	33584	Standard
#		'SM_V7_1H030'	33580	Standard
#		'SM_V7_4H026'	33480	Standard
#		'SM_V7_ZWH024'	33395	Standard
#		'SM_V7_3H014'	33058	Standard
#		'SM_V7_3H013'	33002	Standard
#		'SM_V7_5H020'	32808	Standard
#		'SM_V7_ZWH027'	32768	Standard
#		'S

00:01:39		440000 variants (4995 variants per second), 422530 VCF entries
00:01:41		450000 variants (4986 variants per second), 431965 VCF entries
00:01:43		460000 variants (4988 variants per second), 441404 VCF entries
00:01:45		470000 variants (4990 variants per second), 450925 VCF entries
00:01:47		480000 variants (4985 variants per second), 460470 VCF entries
00:01:49		490000 variants (5002 variants per second), 469964 VCF entries
00:01:51		500000 variants (4998 variants per second), 479496 VCF entries
00:01:53		510000 variants (5005 variants per second), 489149 VCF entries
00:01:55		520000 variants (5004 variants per second), 498766 VCF entries
00:01:57		530000 variants (5015 variants per second), 508390 VCF entries
00:01:58		540000 variants (5023 variants per second), 518087 VCF entries
00:02:00		550000 variants (5022 variants per second), 527594 VCF entries
00:02:02		560000 variants (5021 variants per second), 537323 VCF entries
00:02:04		570000 variants (5030 variants per second

00:04:41		1430000 variants (5283 variants per second), 1379558 VCF entries
00:04:43		1440000 variants (5286 variants per second), 1389333 VCF entries
00:04:45		1450000 variants (5291 variants per second), 1398821 VCF entries
00:04:47		1460000 variants (5294 variants per second), 1408477 VCF entries
00:04:48		1470000 variants (5295 variants per second), 1418221 VCF entries
00:04:50		1480000 variants (5295 variants per second), 1427989 VCF entries
00:04:52		1490000 variants (5296 variants per second), 1437751 VCF entries
00:04:54		1500000 variants (5296 variants per second), 1447564 VCF entries
00:04:56		1510000 variants (5299 variants per second), 1457272 VCF entries
00:04:57		1520000 variants (5302 variants per second), 1467103 VCF entries
00:04:59		1530000 variants (5304 variants per second), 1476726 VCF entries
00:05:01		1540000 variants (5309 variants per second), 1486404 VCF entries
00:05:01	Creating sequences from exon information 'SM_V7_6'
00:05:01	Building sequence tree for chro

00:05:56	Building sequence tree for genome sequences
00:05:56	Done.
00:05:56	Done. Loaded 20 sequences.
00:05:56	Creating sequences from exon information 'SM_V7_U003'
00:05:56	Building sequence tree for chromosome 'SM_V7_U003'
00:05:56	Building sequence tree for genome sequences
00:05:56	Done.
00:05:56	Done. Loaded 27 sequences.
00:05:56	Creating sequences from exon information 'SM_V7_2H007'
00:05:56	Building sequence tree for chromosome 'SM_V7_2H007'
00:05:56	Building sequence tree for genome sequences
00:05:56	Done.
00:05:56	Done. Loaded 49 sequences.
00:05:56	Creating sequences from exon information 'SM_V7_6H001'
00:05:56	Building sequence tree for chromosome 'SM_V7_6H001'
00:05:56	Building sequence tree for genome sequences
00:05:56	Done.
00:05:56	Done. Loaded 8 sequences.
00:05:56		1840000 variants (5326 variants per second), 1777944 VCF entries
00:05:56	Creating sequences from exon information 'SM_V7_5H002'
00:05:56	Building sequence tree for chromosome 'SM_V7_5H002'
00:05:56	Bui

00:05:58	Done.
00:05:58	Done. Loaded 13 sequences.
00:05:58	Creating sequences from exon information 'SM_V7_4H004'
00:05:58	Building sequence tree for chromosome 'SM_V7_4H004'
00:05:58	Building sequence tree for genome sequences
00:05:58	Done.
00:05:58	Done. Loaded 15 sequences.
00:05:58	Creating sequences from exon information 'SM_V7_1H004'
00:05:58	Building sequence tree for chromosome 'SM_V7_1H004'
00:05:58	Building sequence tree for genome sequences
00:05:58	Done.
00:05:58	Done. Loaded 11 sequences.
00:05:58	Creating sequences from exon information 'SM_V7_6H008'
00:05:58	Building sequence tree for chromosome 'SM_V7_6H008'
00:05:58	Building sequence tree for genome sequences
00:05:58	Done.
00:05:58	Done. Loaded 14 sequences.
00:05:58	Creating sequences from exon information 'SM_V7_5H012'
00:05:58	Building sequence tree for chromosome 'SM_V7_5H012'
00:05:58	Building sequence tree for genome sequences
00:05:58	Done.
00:05:58	Done. Loaded 17 sequences.
00:05:59	Creating sequences from 

00:06:00	Done.
00:06:00	Done. Loaded 4 sequences.
00:06:00	Creating sequences from exon information 'SM_V7_1H018'
00:06:00	Building sequence tree for chromosome 'SM_V7_1H018'
00:06:00	Building sequence tree for genome sequences
00:06:00	Done.
00:06:00	Done. Loaded 2 sequences.
00:06:00	Creating sequences from exon information 'SM_V7_3H008'
00:06:00	Building sequence tree for chromosome 'SM_V7_3H008'
00:06:00	Building sequence tree for genome sequences
00:06:00	Done.
00:06:00	Done. Loaded 6 sequences.
00:06:00	Creating sequences from exon information 'SM_V7_1H019'
00:06:00	Building sequence tree for chromosome 'SM_V7_1H019'
00:06:00	Building sequence tree for genome sequences
00:06:00	Done.
00:06:00	Done. Loaded 18 sequences.
00:06:00	Creating sequences from exon information 'SM_V7_4H012'
00:06:00	Building sequence tree for chromosome 'SM_V7_4H012'
00:06:00	Building sequence tree for genome sequences
00:06:00	Done.
00:06:00	Done. Loaded 18 sequences.
00:06:00	Creating sequences from exo

00:06:01	Building sequence tree for genome sequences
00:06:01	Done.
00:06:01	Done. Loaded 14 sequences.
00:06:01	Creating sequences from exon information 'SM_V7_4H027'
00:06:01	Building sequence tree for chromosome 'SM_V7_4H027'
00:06:01	Building sequence tree for genome sequences
00:06:01	Done.
00:06:01	Done. Loaded 5 sequences.
00:06:01	Creating sequences from exon information 'SM_V7_ZWH025'
00:06:01	Building sequence tree for chromosome 'SM_V7_ZWH025'
00:06:01	Building sequence tree for genome sequences
00:06:01	Done.
00:06:01	Done. Loaded 7 sequences.
00:06:01	Creating sequences from exon information 'SM_V7_ZWH029'
00:06:01	Building sequence tree for chromosome 'SM_V7_ZWH029'
00:06:01	Building sequence tree for genome sequences
00:06:01	Done.
00:06:01	Done. Loaded 4 sequences.
00:06:01	Creating sequences from exon information 'SM_V7_3H016'
00:06:01	Building sequence tree for chromosome 'SM_V7_3H016'
00:06:01	Building sequence tree for genome sequences
00:06:01	Done.
00:06:01	Done. 

00:06:02	Building sequence tree for genome sequences
00:06:02	Done.
00:06:02	Done. Loaded 5 sequences.
00:06:02	Creating sequences from exon information 'SM_V7_U015'
00:06:02	Building sequence tree for chromosome 'SM_V7_U015'
00:06:02	Building sequence tree for genome sequences
00:06:02	Done.
00:06:02	Done. Loaded 4 sequences.
00:06:02	Creating sequences from exon information 'SM_V7_2H036'
00:06:02	Building sequence tree for chromosome 'SM_V7_2H036'
00:06:02	Building sequence tree for genome sequences
00:06:02	Done.
00:06:02	Done. Loaded 3 sequences.
00:06:02	Creating sequences from exon information 'SM_V7_6H027'
00:06:02	Building sequence tree for chromosome 'SM_V7_6H027'
00:06:02	Building sequence tree for genome sequences
00:06:02	Done.
00:06:02	Done. Loaded 3 sequences.
00:06:02	Creating sequences from exon information 'SM_V7_ZWH046'
00:06:02	Building sequence tree for chromosome 'SM_V7_ZWH046'
00:06:02	Building sequence tree for genome sequences
00:06:02	Done.
00:06:02	Done. Loade

: 1

In [15]:
# Move file in the right folder
mv -t "$cdir/" snpEff_summary.* snpEff_genes.txt

(PZQ-R_DNA) (PZQ-R_DNA) 

: 1

## QTL analysis

The analysis relies on several steps:
* **SNP filtration**: bi-allelic site only, read depth (DP) of 10 at least, genotype quality (GQ) of 30.
* **Conversion of the genotype data in R/qtl format (csvs)**: only alternatively fixed alleles from the parent retained.
* **Single QTL analysis on all the phenotypes** (estimated run time: 30 minutes): sum of cercariae shed by schistosome genotype, average of cercariae shed by schistosome genotype, total phenoloxidase activity of the snail host, hemoglobin rate of the snail host.
* **QTL interaction analysis using reduced set of markers** (estimated run time: 12h).
* **Allele dominance analysis.**

N.B.: Estimated run time indicated is dependent on computer/server specifications. Our analysis ran on a server with 16 CPU (2.53 GHz) and 96 GB of RAM.

In [ ]:
Rscript scripts/Cercariae_intensity_qtl.R
Rscript scripts/Fig4_plot_dominance_chr.R

In [2]:
echo a

a
(PZQ-R_DNA) 

: 1

## Prioritization of candidate genes

### Chromosome level VCF

QTLs were identified on chromosomes 1, 2, 3, 4 and 5. The VCF file needs to be reduced to chromosome intervals to avoid loading unnecessary data and saving time.

In [ ]:
gatk SelectVariants \
        -V "data/calling/cerc_prod_snpEff.vcf.gz" \
        -L SM_V7_1:40000000-55000000 \
        -L SM_V7_2:40000000-48130368 \ # End of the chromosome
        -L SM_V7_3:3000000-5500000 \
        -L SM_V7_4:20000000-40000000 \
        -L SM_V7_5:18000000-25000000 \
        -O "data/calling/cerc_prod_snpEff_1-2-3-4-5.vcf.gz"

### Candidate gene analysis

The 5 QTLs identified cover several Mb and tens of genes. To identify the genes most likely involved in cercarial production, we focus on few criteria:
* We refine the QTL regions by determining the chromosomal positions corresponding to a drop of 1.8 LOD from the highest LOD score of each QTL. We expect the genes involved to be relatively close to the peaks.
* We focus on alternative fix alleles in the parents (with a GQ greater than 29 and a DP greater than 5). The phenotype in the parental population being dramatic, we expect to the parents to be homozygous for the alleles involved.
* We take in account mutations and their impacts in each gene. We expect at least on the parental allele to show mutations with important impacts.
* We final take in account know gene expression in late sporocyst and cercariae. We expect the genes to be expressed at these stages.

Using these criteria, we created a formula to generate a score (genoscore) to objectively prioritize candidate genes in each QTL region:

Genoscore = $\sum_{i=1}^{n} (GSa_i \times GSb_i \times IS_i) \over l \times \sqrt(d_peak))$$ \times e \times 100$

The genoscore is computed either by gene or gene’s coding sequence (CDS). For each site i of a gene or CDS, we first compute a genotype score (GS) for each cross a and b by associating a score of 20 if genotypes are alternatively fixed in each cross parent or a score of 5 if fixed in one parent and unknown in the other (any other genotype combination received 0) and an impact score (IS) from the snpEff annotation (low impact or modifier mutation = 1, moderate impact = 5, high impact = 10). We sum the results for the n sites present in the gene or CDS. We weigh this sum using the length l of the gene or the CDS, and the square root of the distance d between the position of the highest LOD score (peak) and the start of the gene or the CDS. This was finally modulated by an expression factor e of 1 if expressed or -1 otherwise. Jupyter notebook and scripts used to obtain gene expression are available on [Github](https://github.com/fdchevalier/Sm_gene_expression).

In [ ]:
Rscript scripts/Candidate_genes.R

## Copy number variation

In [ ]:
for i in $(find data/libraries/1-GWAS/ -name "*_sorted_MD_recal.bam" | sort)
do
    sample=$(basename $(dirname "$i"))
    output="$(dirname "$i")/${sample}_3_QTL.cov"
    echo "samtools view -u $i SM_V7_3 | bedtools genomecov -ibam - -d | grep -w SM_V7_3 | awk '\$3 > 0 {print}' > $output" | qsub -V -cwd -o "$statdir" -j y -r y -pe smp 12
done

In [2]:
for i in $(find data/libraries/2-ER-ES_populations/ -name "*_sorted_MD_recal.bam" | sort)
do
    sample=$(basename $(dirname "$i"))
    output="$(dirname "$i")/${sample}_3_QTL.cov"
    echo "samtools view -u $i SM_V7_3 | bedtools genomecov -ibam - -d | grep -w SM_V7_3 | awk '\$3 > 0 {print}' > $output" | qsub -V -cwd -o "$statdir" -j y -r y -pe smp 12
done

bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
bash: qsub: command not found...
(PZQ-R_DNA) 

: 1

## SmTRP mutations

### GWAS

In [17]:
for i in $(find data/libraries/1-GWAS -maxdepth 1 -printf '%P\n' | sed "/^$/d")
do
    echo -e "$i\t$(cut -d "_" -f 3 <<< "$i")"
done | sort -k2 > data/pop_GWAS

(PZQ-R_DNA) 

: 1

In [14]:
gdir="data/genome"
cdir="data/calling"
myvcf="$cdir/$(grep -m 1 VCF_PFX GWAS.smk | cut -d "\"" -f 2).vcf.gz"

(PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) 

: 1

In [8]:
mutdir="results/4-Mutations in SmTRP/1-GWAS"
[[ ! -d "$mutdir" ]] && mkdir -p "$mutdir"

(PZQ-R_DNA) (PZQ-R_DNA) 

: 1

In [16]:
scripts/func-table-mut.sh -v "$myvcf" \
    -r "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"\
    -g "$gdir/Smp_246790.5.gff" \
    -p data/pop_GWAS \
    -o "$mutdir/Smp_246790.tsv"

scripts/func-table-mut.sh: line 246: trap: in: invalid signal specification
scripts/func-table-mut.sh: line 246: trap: SmTRP/Smp_246790.tsv results/4-Mutations: invalid signal specification
scripts/func-table-mut.sh: line 246: trap: in: invalid signal specification
scripts/func-table-mut.sh: line 246: trap: SmTRP/[na]_Smp_246790_cds.fa results/4-Mutations: invalid signal specification
scripts/func-table-mut.sh: line 246: trap: in: invalid signal specification
scripts/func-table-mut.sh: line 246: trap: SmTRP/[aa]_Smp_246790.fa: invalid signal specification
(PZQ-R_DNA) 

: 1

### Selected populations

In [20]:
for i in $(find data/libraries/2-ER-ES_populations -maxdepth 1 -printf '%P\n' | sed "/^$/d")
do
    echo -e "$i\t$(cut -d "_" -f 1 <<< "$i")"
done | sort -k2 > data/pop_ER-ES

(PZQ-R_DNA) 

: 1

In [21]:
gdir="data/genome"
cdir="data/calling"
myvcf="$cdir/$(grep -m 1 VCF_PFX Selected_populations.smk | cut -d "\"" -f 2).vcf.gz"

(PZQ-R_DNA) (PZQ-R_DNA) (PZQ-R_DNA) 

: 1

In [23]:
mutdir="results/4-Mutations in SmTRP/2-ER-ES_populations"
[[ ! -d "$mutdir" ]] && mkdir -p "$mutdir"

(PZQ-R_DNA) (PZQ-R_DNA) 

: 1

In [24]:
scripts/func-table-mut.sh -v "$myvcf" \
    -r "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"\
    -g "$gdir/Smp_246790.5.gff" \
    -p data/pop_ER-ES \
    -o "$mutdir/Smp_246790.5.tsv"

Info: Identifying samples related to SmLE-PZQ-ER, SmLE-PZQ-ES population(s)
Info: Extraction information from GFF
Info: Generating reference sequence
sed: couldn't write 518 items to stdout: Broken pipe
Progress: [========================================] 100%
(PZQ-R_DNA) 

: 1

In [22]:
echo $myvcf

data/calling/PZQ_ER-ES.vcf.gz
(PZQ-R_DNA) 

: 1